<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_Simple_Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.0.0-beta-1

     |████████████████████████████████| 87.9MB 46kB/s 
     |████████████████████████████████| 501kB 40.8MB/s 
     |████████████████████████████████| 3.1MB 51.6MB/s 


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
%%capture

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import InputLayer, LSTM, GRU, Input, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.layers import Dense, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import Model

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


###removing all items that are from the original jigsaw bias data from Kaggle to small_train.csv, which only has 100 and below text

In [6]:
'''
!unzip train.csv

data = pd.read_csv  ('train.csv')


np.max (text_column.apply(len)), data.shape

from nltk.corpus import stopwords
import nltk
import re

nltk.download ('stopwords')

text = text_column.apply (clean_text)

small = list (text_column[~(text_column.map(len) <= 100)].index)
small_data = data.iloc [small]
small_data.to_csv ('small_train.csv')

Archive:  train.csv.zip
  inflating: train.csv               


In [19]:
from nltk.corpus import stopwords
import nltk
import re

nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
test_stop = stopwords.words ('english')

In [0]:
def clean_text(x):
    x = x.lower()
    x = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)
    stop_words = stopwords.words ('english')
    words = re.sub(r'[^\w\s]','',x).split(' ')
    words = [word for word in words if word not in stop_words]
    return words

In [23]:
data = pd.read_csv ('small_train.csv')
data.head(3)

,Unnamed: 0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,8,239575,0.0,The ranchers seem motivated by mostly by greed...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-13 18:01:05.156229+00,6,NaN,26662,approved,0,0,0,0,0,0.0,0,4


In [0]:
text_column = data['comment_text']
text = text_column.apply (clean_text)

In [0]:
under_100 = text.apply(len) <= 100

In [0]:
flags = ['severe_toxicity', 'obscene','identity_attack', 'insult', 'threat']

In [0]:
MAX_SEQ_LENGTH = 100
MAX_VOCAB_SIZE = 30000
EMBEDDING_DIM = 50

In [0]:
text = text[under_100].values

In [0]:
targets = data[flags][under_100].values

In [65]:
text.shape, targets.shape

((1299202,), (1299202, 5))

In [0]:
X_train, X_test, y_train, y_test = train_test_split (text, targets, random_state=23, test_size=0.2)

In [69]:
X_train.shape, X_test.shape

((1039361,), (259841,))

In [0]:
tokenizer = Tokenizer (num_words = MAX_VOCAB_SIZE)

In [0]:
tokenizer.fit_on_texts (X_train)

In [0]:
X_train_seq = tokenizer.texts_to_sequences (X_train)
X_test_seq = tokenizer.texts_to_sequences (X_test)

In [0]:
X_train_seq_pad = pad_sequences (X_train_seq, maxlen=MAX_SEQ_LENGTH, padding='post')
X_test_seq_pad = pad_sequences (X_test_seq, maxlen=MAX_SEQ_LENGTH, padding='post')

In [0]:
word_to_idx = tokenizer.word_index
V = len (word_to_idx) # (V)ocab

In [0]:
T = X_train_seq_pad.shape[1]

In [0]:
D = 32

In [0]:
i = Input (shape=(T,))

x = Embedding (V+1, D) (i)
x = Dropout (0.5) (x)
x = Bidirectional (LSTM (128, return_sequences=True)) (x)
x = BatchNormalization () (x)
x = GlobalMaxPooling1D () (x)
x = Dropout (0.5) (x)
x = Dense (16, activation='relu') (x)
x = Dense (len(flags), activation='softmax' ) (x)

In [0]:
model = Model (i, x)

In [0]:
adam = tf.keras.optimizers.Adam (learning_rate=0.05, decay=5e-5)

In [0]:
model.compile (optimizer=adam,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
epochs = 1

r = model.fit (X_train_seq_pad, y_train, epochs=epochs, batch_size=64)

Train on 1039361 samples
   7680/1039361 [..............................] - ETA: 1:46:12 - loss: 0.8256 - accuracy: 0.2059